# Notebook imports

In [1]:
from os import walk
from os.path import join

import pandas as pd

# Constants

In [10]:
EXAMPLE_FILE = 'SpamData/01_Processing/practice_email.txt'
SPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_1/'
SPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_2/'
NON_SPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1/'
NON_SPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2/'
SPAM = 1
NON_SPAM = 0

# Reading Files

In [3]:
file = open(EXAMPLE_FILE, encoding='latin-1')
is_body = False
lines = []
for line in file:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True
file.close()

email_body = '\n'.join(lines)

# Generator Functions

In [4]:
def generate_square(n):
    for number in range(n):
        yield number ** 2

generate_square(10)

<generator object generate_square at 0x000002061B3C8648>

In [5]:
for i in generate_square(5):
    print(i, end=' ')

# สาเหตุที่ต้องใช้ก็คือมันมีการจำ state และเมื่อเราเล่นกับ dataset ใหญ่ๆ
# ลองนึกภาพว่าคำนวณก้อนเท่าบ้านออกมาอะ ขนาดข้อมูลนะ ใช้เวลานานมากด้วยกว่าจะรัน state ถัดไปได้

0 1 4 9 16 

# Email body extraction

In [6]:
def email_body_generator(path):
    for root, dir_names, file_names in walk(path):
        for file_name in file_names:
            file = open(join(root, file_name), encoding='latin-1')
            is_body = False
            lines = []
            for line in file:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True
            file.close()
            email_body = '\n'.join(lines)
            yield file_name, email_body 

In [7]:
def get_df_from_directory(path, classification):
    rows = []
    row_names = []
    for file_name, email_body in email_body_generator(path):
        rows.append({'MESSAGE': email_body, 'CATEGORY': classification})
        row_names.append(file_name)
    return pd.DataFrame(rows, index=row_names)

In [11]:
spam_emails = get_df_from_directory(SPAM_1_PATH, SPAM)
spam_emails = spam_emails.append(get_df_from_directory(SPAM_2_PATH, SPAM))
spam_emails = spam_emails.append(get_df_from_directory(NON_SPAM_1_PATH, NON_SPAM))
spam_emails = spam_emails.append(get_df_from_directory(NON_SPAM_2_PATH, NON_SPAM))
spam_emails

,MESSAGE,CATEGORY
00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",1
00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,1
00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,1
00004.eac8de8d759b7e74154f142194282724,##############################################...,1
00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,1
...,...,...
01396.61983fbe6ec43f55fd44e30fce24ffa6,http://news.bbc.co.uk/1/hi/england/2515127.stm...,0
01397.9f9ef4c2a8dc012d80f2ce2d3473d3b7,"> >-- be careful when using this one.) Also, t...",0
01398.169b51731fe569f42169ae8f948ec676,">>>>> ""SM"" == Skip Montanaro <skip@pobox.com> ...",0
01399.ca6b00b7b341bbde9a9ea3dd6a7bf896,"So then, ""Mark Hammond"" <mhammond@skippinet.co...",0
